**Installing OpenAI CLIP**

In [ ]:
pip install git+https://github.com/openai/CLIP.git

**Import Libraries**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from PIL import Image
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import segmentation_models_pytorch as smp
from torch.nn.functional import cosine_similarity
import clip

**Load and Freeze CLIP Model**

In [ ]:
# Initialize CLIP model and the preprocess function for the images

device = "cuda" if torch.cuda.is_available() else "cpu"

clip_model, preprocess = clip.load("RN101", device=device)

# Freeze CLIP model parameters (no training required)

for param in clip_model.parameters():
    param.requires_grad = False

**Loading Affordance Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Define Paths to Dataset Directories**

In [ ]:
train_images_dir = '/content/drive/My Drive/AffordanceDatasetMain/Train_Set/Images'
train_masks_dir = '/content/drive/My Drive/AffordanceDatasetMain/Train_Set/Masks'
train_annotations_path = '/content/drive/My Drive/AffordanceDatasetMain/Train_Set/annotations.json'

In [ ]:
test_images_dir = '/content/drive/My Drive/AffordanceDatasetMain/Test_Set/Images'
test_annotations_path = '/content/drive/My Drive/AffordanceDatasetMain/Test_Set/annotations.json'

**Load annotations from the JSON file**

In [ ]:
def load_annotations(annotation_path):
    with open(annotation_path, 'r') as f:
        annotations = json.load(f)
    return annotations

train_annotations = load_annotations(train_annotations_path)
test_annotations = load_annotations(test_annotations_path)

**Training Dataset Class**

In [ ]:
class AffordanceDataset(Dataset):
    def __init__(self, annotations, images_dir, masks_dir, transform=None):
        self.annotations = annotations
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        self.preprocess = preprocess

    def load_image(self, image_path):
        image = Image.open(image_path).convert('RGB')
        return image

    def load_mask(self, mask_path):
        mask = Image.open(mask_path).convert('L')
        if self.transform:
            mask = self.transform(mask)
        return mask

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        annotation = self.annotations[idx]
        query = annotation['query']
        object_type = annotation['object_type']
        image_file = annotation['file_name']
        mask_file = annotation['mask_file_name']

        queries = query + " " + object_type

        image_path = os.path.join(self.images_dir, query, object_type, image_file)
        mask_path = os.path.join(self.masks_dir, query, object_type, mask_file)

        image = self.load_image(image_path)
        mask = self.load_mask(mask_path)

        image = self.preprocess(image)

        return image, queries, mask

**Train Dataset Creation**

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = AffordanceDataset(
    annotations=train_annotations,
    images_dir=train_images_dir,
    masks_dir=train_masks_dir,
    transform=transform
)

**Create DataLoader for Batching**

In [ ]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

**Feature Extraction using CLIP**

In [ ]:
def extract_features(image, query):

    # Preprocess images and tokenize queries
    text_inputs = clip.tokenize(query).to(device)

    # Extract Text Features and Image Features
    with torch.no_grad():
        text_features = clip_model.encode_text(text_inputs).to(torch.float32)
        image_features = clip_model.encode_image(image).to(torch.float32)
        #image_features = self.clip_model.visual(image).to(torch.float32)

    return image_features, text_features

**Binary Segmentation UNet model**

In [ ]:
# Use UNet from segmentation_models_pytorch, with a ResNet-101 Encoder
Unet_model = smp.Unet(
    encoder_name="resnet101",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1
).to(device)

# Freeze the encoder to keep the pre-trained weights intact
for param in Unet_model.encoder.parameters():
    param.requires_grad = False

**AffordanceCLIP Model: Complete Model**

In [ ]:
class UNetWithCLIP(nn.Module):
    def __init__(self, clip_model, unet_model):
        super(UNetWithCLIP, self).__init__()
        self.clip_model = clip_model
        self.unet_model = unet_model
        self.extract_features = extract_features #
        self.fusion_layer = nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1, stride=1)

    def forward(self, image, query, processed_image):

        # 1. Extract CLIP features (image and query)
        image_features, text_features = self.extract_features(processed_image, query)

        # 2. Pass the image through UNet to get the segmentation mask
        segmentation_output = self.unet_model(image)

        # 3. Fuse CLIP image features with UNet output
        image_features = image_features.unsqueeze(2).unsqueeze(3)
        image_features = image_features.expand_as(segmentation_output)

        concatenated_features = torch.cat((segmentation_output, image_features), dim=1)

        # 4. Pass concatenated features through a fusion layer
        refined_output = self.fusion_layer(concatenated_features)

        # 5. Compute similarity score between image and query features
        similarity_score = cosine_similarity(image_features.squeeze(), text_features)

        # 6. Use similarity score to modulate the segmentation mask
        final_mask = refined_output * similarity_score.unsqueeze(-1).unsqueeze(-1)

        return final_mask

In [ ]:
# Model Creation
model = UNetWithCLIP(clip_model, unet_model).to(device)

In [ ]:
# Define Binary Cross Entropy
criterion = nn.BCELoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    total_pixel_accuracy = 0.0
    total_iou = 0.0
    total_dice = 0.0
    total_batches = len(train_loader)

    for batch_idx, (images, queries, masks) in enumerate(train_loader):
        images = images.to(device)
        masks = masks.to(device)

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass (predict affordance maps)
        affordance_maps = model(images, queries)

        # Apply sigmoid to affordance_maps to ensure values are in the range [0, 1]
        affordance_maps = torch.sigmoid(affordance_maps)

         # Compute the BCE + Dice loss
        loss = bce_dice_loss(affordance_maps, masks, bce_weight=0.5)

        # Compute the loss
        # loss = criterion(affordance_maps, masks)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Accumulate the loss
        epoch_loss += loss.item()

        # Detach predicted maps and masks for evaluation (to avoid gradients being tracked)
        affordance_maps = affordance_maps.detach()
        masks = masks.detach()

        # Compute Pixel-wise Accuracy, IoU, and Dice Coefficient
        batch_pixel_accuracy = pixel_accuracy(affordance_maps, masks)
        batch_iou = iou(affordance_maps, masks)
        batch_dice = dice_coefficient(affordance_maps, masks)

        # Accumulate metrics
        total_pixel_accuracy += batch_pixel_accuracy
        total_iou += batch_iou
        total_dice += batch_dice

    # Compute average loss and metrics for the epoch
    avg_loss = epoch_loss / total_batches
    avg_pixel_accuracy = total_pixel_accuracy / total_batches
    avg_iou = total_iou / total_batches
    avg_dice = total_dice / total_batches

    # Print results for the current epoch
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Train Loss: {avg_loss:.4f}, Pixel Accuracy: {avg_pixel_accuracy:.4f}, IoU: {avg_iou:.4f}, Dice Coefficient: {avg_dice:.4f}")
